# Getting Paragraph predictions as Annotations in the smartview

This notebook provides a tutorial for using automatic paragraph prediction and enabling the predictions in smartview mode.

Importing necessary dependencies:

In [1]:
import requests

from konfuzio_sdk import KONFUZIO_TOKEN
from konfuzio_sdk.data import Project, Annotation, Span, Document
from tqdm import tqdm
from typing import List, Dict

Next, we need to specify project ID and load its documents' IDs for further usage:

In [2]:
PROJECT_ID = 1641

In [3]:
my_project = Project(id_=PROJECT_ID, update=True)

2022-09-16 13:26:59,393 [konfuzio_sdk.data   ] [MainThread] [INFO    ] [init_or_update_docum][2489] Document 1 Lorem ipsum.pdf (338280) was updated, we will download it again as soon you use it.


Before sending requests to the API we need to specify the credentials and include them into headers used in the request:

In [4]:
headers = {
    "Authorization": "Token {}".format(KONFUZIO_TOKEN)
}

For each document, we send a GET request to Konfuzio's API and receive a JSON with coordinates of paragraphs. These coordinates need to be transformed with accordance to the image rescaling because sizes of actual document images and resulting pages are different; let's define a function that does so:

In [5]:
def convert_bbox_segmentation_to_konfuzio(bbox_pages: List[List[Dict]], doc: Document) -> List[List[Dict]]:
    page = doc.pages()[0]
    img = page.get_image(update=True)
    factor_x = page.width / img.width # determines difference between sizes of page and actual document images
    factor_y = page.height / img.height
    new_bbox_pages = []
    for bboxes in bbox_pages:
        new_bboxes = []
        for bbox in bboxes:
            new_bbox = {
                'x0': factor_x * bbox['x0'], # rescaling the obtained coordinates 
                'x1': factor_x * bbox['x1'],
                'y0': factor_y * (img.height - bbox['y1']),
                'y1': factor_y * (img.height - bbox['y0']),
                'label': bbox['label']
            }
            new_bboxes.append(new_bbox)
        new_bbox_pages.append(new_bboxes)
    return new_bbox_pages

In [6]:
paragraph_coords = {}

for doc in tqdm(my_project._documents):
    r = requests.get('https://app.konfuzio.com/api/projects/{}/docs/{}/segmentation/'.format(PROJECT_ID,
                                                                                            doc.id_),
                     headers=headers
                    )
    
    paragraph_coords[doc.id_] = convert_bbox_segmentation_to_konfuzio(r.json(), doc)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:09<00:00,  1.56s/it]


Let's run an exemplary annotation for one of the documents from the test set:

In [7]:
EXAMPLE_DOC_ID = 338280
example_doc = my_project.get_document_by_id(EXAMPLE_DOC_ID)

Gathering all bounding boxes from all pages to iterate over the resulting list later:

In [8]:
detections = []

for i, page in enumerate(paragraph_coords[EXAMPLE_DOC_ID]):
    for j, bbox in enumerate(page):
        detections.append((i,j))

Filtering over the character-level bounding boxes to determine those falling into different segments:

In [ ]:
for page_i, bbox_i in detections:
    bboxes_within_paragraph = {
        int(k): b for k, b in example_doc.pages()[page_i].get_bbox().items() if
        (
            (b['x0'] > paragraph_coords[EXAMPLE_DOC_ID][page_i][bbox_i]['x0']) and
            (b['x1'] < paragraph_coords[EXAMPLE_DOC_ID][page_i][bbox_i]['x1']) and
            (b['y0'] > paragraph_coords[EXAMPLE_DOC_ID][page_i][bbox_i]['y0']) and
            (b['y1'] < paragraph_coords[EXAMPLE_DOC_ID][page_i][bbox_i]['y1'])
        )
    }
    
    # creating a new Annotation for each segment defined
    
    label = my_project.get_label_by_name(paragraph_coords[EXAMPLE_DOC_ID][page_i][bbox_i]['label'])
    new_ann = Annotation(
        document=example_doc,
        spans=[Span(start_offset=min(bboxes_within_paragraph.keys()), end_offset=max(bboxes_within_paragraph.keys())+1)],
        label=label,
        label_set=label.label_sets[0],
        accuracy=1.0,
        is_correct=True
    )
    
    # saving the annotation online
    
    new_ann.save()
    
    # refresh https://app.konfuzio.com/projects/1641/docs/338280/bbox-annotations/ to see it